In [38]:
import pandas as pd
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter

# CSVの読み込み
df = pd.read_csv("warehouse_shop_randomized_limited.csv")

# ジオコーダーの初期化
geolocator = Nominatim(user_agent="geoapi_exercise")
geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1)

# warehouseとshopの一意な地点一覧を作成
unique_locations = pd.Series(pd.concat([df['warehouse'], df['shop']]).unique(), name="location")

# 緯度経度の取得
location_coords = unique_locations.apply(geocode)
latitudes = location_coords.apply(lambda loc: loc.latitude if loc else None)
longitudes = location_coords.apply(lambda loc: loc.longitude if loc else None)

# 緯度経度のDataFrame作成
coords_df = pd.DataFrame({
    "location": unique_locations,
    "latitude": latitudes,
    "longitude": longitudes
})

# 元データにマージ
df = df.merge(coords_df, how='left', left_on='warehouse', right_on='location')
df = df.rename(columns={"latitude": "warehouse_lat", "longitude": "warehouse_lon"}).drop(columns=["location"])

df = df.merge(coords_df, how='left', left_on='shop', right_on='location')
df = df.rename(columns={"latitude": "shop_lat", "longitude": "shop_lon"}).drop(columns=["location"])

# 結果確認
print(df.head())

# 保存（必要に応じて）
df.to_csv("warehouse_shop_with_coordinates.csv", index=False)

  warehouse  shop  value  warehouse_lat  warehouse_lon   shop_lat    shop_lon
0       佐賀市   山口市    754      33.263913     130.300838  34.178132  131.473708
1       佐賀市   新潟市    214      33.263913     130.300838  37.916376  139.036506
2       佐賀市   松山市    125      33.263913     130.300838  33.839519  132.765352
3       佐賀市   宮崎市    859      33.263913     130.300838  31.907633  131.420402
4       佐賀市  名古屋市    381      33.263913     130.300838  35.185105  136.899844
